In [42]:
spark.stop()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName('Project') \
        .config("spark.master", "yarn") \
        .config("spark.executor.instances", "4") \
        .config("spark.executor.cores", "1") \
        .config("spark.executor.memory", "1g") \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/01 14:45:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/01 14:45:55 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


# zitoumeno 4 rdd by dimitris

In [47]:
spark.stop()
sc.stop()

In [46]:
from pyspark import SparkContext

# Αρχικοποίηση Spark Context
sc = SparkContext("local[4]", "RDD Query Example")

# φόρτοση δεδομένων και ένωση
import csv
rdd1 = spark.textFile("hdfs://okeanos-master:54310/user/project/Crime_Data_from_2010_to_2019.csv") \
    .map(lambda x: next(csv.reader([x])))
header1 = rdd1.first()
rdd1 = rdd1.filter(lambda row: row != header1)
rdd2 = spark.textFile("hdfs://okeanos-master:54310/user/project/Crime_Data_from_2020_to_Present.csv") \
    .map(lambda x: next(csv.reader([x])))   
header2 = rdd2.first()  
rdd2 = rdd2.filter(lambda row: row != header2)
rdd = rdd1.union(rdd2)

#querry execution
rdd = rdd.map(lambda col: (col[3], col[14]))

AttributeError: 'SparkSession' object has no attribute 'textFile'

In [10]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("RDD query") \
    .getOrCreate() \
    .sparkContext

24/01/01 15:44:40 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [48]:
from pyspark import SparkContext
sc = SparkContext("local[4]", "RDD Query Example")


import csv

# Load the first CSV file into an RDD
rdd1 = sc.textFile("hdfs://okeanos-master:54310/user/project/Crime_Data_from_2010_to_2019.csv") \
    .map(lambda x: next(csv.reader([x])))
    
header1 = rdd1.first()
rdd1 = rdd1.filter(lambda row: row != header1)

# Load the second CSV file into an RDD
rdd2 = sc.textFile("hdfs://okeanos-master:54310/user/project/Crime_Data_from_2020_to_Present.csv") \
    .map(lambda x: next(csv.reader([x])))

    
header2 = rdd2.first()  
rdd2 = rdd2.filter(lambda row: row != header2)

# Merge the two RDDs
rdd = rdd1.union(rdd2)

#query execution
rdd = rdd.map(lambda col: (col[3], col[14])).filter(lambda x: x[1] == '101').map(lambda x: (x[0])).map(lambda x: ('morning') if 500 <= x < 1200 else 
                          (('afternoon') if 1200 <= x < 1700 else 
                           (('evening') if 1700 <= x < 2100 else 
                            ('night')))).map(lambda x: (x,1)).reduceByKey(lambda a,b: a + b).sortBy(lambda x: x[1], ascending=False)

print(rdd.collect())

24/01/01 17:12:38 WARN BlockReaderFactory: I/O error constructing remote block reader.
java.io.InterruptedIOException: Interrupted while waiting for IO on channel java.nio.channels.SocketChannel[connected local=/192.168.0.1:54450 remote=/192.168.0.1:9866]. Total timeout mills is 60000, 60000 millis timeout left.
	at org.apache.hadoop.net.SocketIOWithTimeout$SelectorPool.select(SocketIOWithTimeout.java:350)
	at org.apache.hadoop.net.SocketIOWithTimeout.doIO(SocketIOWithTimeout.java:156)
	at org.apache.hadoop.net.SocketInputStream.read(SocketInputStream.java:161)
	at org.apache.hadoop.net.SocketInputStream.read(SocketInputStream.java:131)
	at org.apache.hadoop.net.SocketInputStream.read(SocketInputStream.java:118)
	at java.base/java.io.FilterInputStream.read(FilterInputStream.java:83)
	at org.apache.hadoop.hdfs.protocolPB.PBHelperClient.vintPrefixed(PBHelperClient.java:519)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderRemote.newBlockReader(BlockReaderRemote.java:414)
	at org.apache.

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (okeanos-master executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/user/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/home/user/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1237, in process
    out_iter = func(split_index, iterator)
  File "/home/user/.local/lib/python3.10/site-packages/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/user/.local/lib/python3.10/site-packages/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/user/.local/lib/python3.10/site-packages/pyspark/rdd.py", line 

In [26]:
#keep only the elements with premis cd == 101
rdd = rdd.filter(lambda x: x[1] == '101')

In [27]:
#kratame mono to time occ
rdd = rdd.map(lambda x: (x[0]))

In [28]:
#metatropi se int
rdd = rdd.map(lambda x: int(x))

In [29]:
rdd = rdd.map(lambda x: ('morning') if 500 <= x < 1200 else 
                          (('afternoon') if 1200 <= x < 1700 else 
                           (('evening') if 1700 <= x < 2100 else 
                            ('night'))))


In [31]:
rdd = rdd.map(lambda x: (x,1))

In [33]:
rdd = rdd.reduceByKey(lambda a,b: a + b)

In [35]:
rdd = rdd.sortBy(lambda x: x[1], ascending=False)

In [ ]:
print(rdd.collect())

In [ ]:
print(rdd.first())

# zitoumeno 4 df by dimitris

In [25]:
spark.stop()

In [1]:
from pyspark.sql import SparkSession

# Αρχικοποίηση Spark Session
spark = SparkSession.builder \
    .appName("DataFrame Query Example") \
    .master("local[4]") \
    .getOrCreate()

# Read the CSVs file into a DataFrames
df1 = spark.read.csv('hdfs://okeanos-master:54310/user/project/Crime_Data_from_2010_to_2019.csv', header=True, inferSchema=False).select("TIME OCC","Premis Cd")
df2 = spark.read.csv('hdfs://okeanos-master:54310/user/project/Crime_Data_from_2020_to_Present.csv', header=True, inferSchema=False).select("TIME OCC","Premis Cd")

df = df1.union(df2)

#ksekiname to xrono
import time
start_time = time.time()

#imports gia query execution
from pyspark.sql.functions import col
from pyspark.sql.functions import when

##query execution
df = df.filter(df['Premis Cd'] == '101').select("TIME OCC").withColumn("TIME OCC", col("TIME OCC").cast("int"))

df = df.withColumn(
    "TimeOfDay",
    when((df["TIME OCC"] >= 500) & (df["TIME OCC"] < 1200), "morning")
    .otherwise(
        when((df["TIME OCC"] >= 1200) & (df["TIME OCC"] < 1700), "afternoon")
        .otherwise(
            when((df["TIME OCC"] >= 1700) & (df["TIME OCC"] < 2100), "evening")
            .otherwise("night")
        )
    )
)

df = df.select("TimeOfDay").groupBy("TimeOfDay").count().orderBy("TimeOfDay")

#stamatame xrono
end_time = time.time()\

#apotelesmata
df.show()
print("Dataframe API Execution Time: ", end_time - start_time)


spark.stop()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/04 17:14:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/user/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/user/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

# zitoumeno 5 df by dimitris

In [5]:
spark.stop()

In [13]:
from pyspark.sql import SparkSession

# Αρχικοποίηση Spark Session
spark = SparkSession.builder \
    .appName("DataFrame Query Example") \
    .master("local[4]") \
    .getOrCreate()

# Read the CSVs file into a DataFrames
df1 = spark.read.csv('hdfs://okeanos-master:54310/user/project/Crime_Data_from_2010_to_2019.csv', header=True, inferSchema=False)
df2 = spark.read.csv('hdfs://okeanos-master:54310/user/project/Crime_Data_from_2020_to_Present.csv', header=True, inferSchema=False)
df = df1.union(df2).select('Date Rptd', 'Vict Descent','LAT', 'LON')

from pyspark.sql.functions import col
df = df.filter(col('Vict Descent').isNotNull())

##kratame mono ta egklimata toy 2015
from pyspark.sql.functions import col, to_date
from pyspark.sql.functions import year

df = df.withColumn("Date Rptd", to_date(col("Date Rptd"), 'MM/dd/yyyy hh:mm:ss a'))
df = df.withColumn("Year", year("Date Rptd")).drop("Date Rptd").filter(col("Year") == 2015).drop('Year')

# Example: UDF to calculate square of a number
def get_descent(code):
    descents = {
        "A": "Other Asian",
        "B": "Black",
        "C": "Chinese",
        "D": "Cambodian",
        "F": "Filipino",
        "G": "Guamanian",
        "H": "Hispanic/Latin/Mexican",
        "I": "American Indian/Alaskan Native",
        "J": "Japanese",
        "K": "Korean",
        "L": "Laotian",
        "O": "Other",
        "P": "Pacific Islander",
        "S": "Samoan",
        "U": "Hawaiian",
        "V": "Vietnamese",
        "W": "White",
        "X": "Unknown",
        "Z": "Asian Indian"
    }
    return descents.get(code, "Invalid Code")  # Default to "Invalid Code" if not found

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType


get_descent_udf = udf(get_descent, StringType())

from pyspark.sql.functions import col
df = df.withColumn("Vict Descent", get_descent_udf(col("Vict Descent")))

df = df.withColumnRenamed("LAT", "LAT_df").withColumnRenamed("LON", "LON_df").withColumnRenamed("Vict Descent", "Vict_Descent")


from pyspark.sql.functions import col, regexp_replace

income = spark.read.csv('hdfs://okeanos-master:54310/user/project/income/LA_income_2015.csv', header=True, inferSchema=True)
income = income.withColumn("Estimated Median Income", regexp_replace("Estimated Median Income", "[^0-9]", "").cast("int"))

income = income.select("Zip Code","Estimated Median Income")
top_3_income = income.orderBy(col("Estimated Median Income").desc()).limit(4)

least_3_income = income.orderBy(col("Estimated Median Income").asc()).limit(4)
income = top_3_income.union(least_3_income)

top_3_income = top_3_income.withColumnRenamed("Zip Code", "zipcode_income").drop("Estimated Median Income")
least_3_income = least_3_income.withColumnRenamed("Zip Code", "zipcode_income").drop("Estimated Median Income")
income = income.withColumnRenamed("Zip Code", "zipcode_income").drop("Estimated Median Income")


top_3_income = top_3_income.filter(top_3_income.zipcode_income != "91210")
least_3_income = least_3_income.filter(least_3_income.zipcode_income != "91046")

geocoding = spark.read.csv('hdfs://okeanos-master:54310/user/project/revgecoding.csv', header=True, inferSchema=True).withColumnRenamed("LAT", "LAT_geocoding").withColumnRenamed("LON", "LON_geocoding").withColumnRenamed("ZIPcode", "zipcode_geocoding")


join_1 = df.join(geocoding, (df.LAT_df == geocoding.LAT_geocoding)&(df.LON_df == geocoding.LON_geocoding))

join_1 = join_1.drop("LAT_df","LAT_geocoding","LON_df","LON_geocoding")

join_2 = join_1.join(least_3_income, (join_1.zipcode_geocoding == least_3_income.zipcode_income))

join_2 = join_2.select("Vict_Descent")
join_2 = join_2.groupBy("Vict_Descent").count().orderBy(col("count").desc())

join_3 = join_1.join(top_3_income, (join_1.zipcode_geocoding == top_3_income.zipcode_income))
join_3 = join_3.select("Vict_Descent")
join_3 = join_3.groupBy("Vict_Descent").count().orderBy(col("count").desc())

In [14]:
join_2.orderBy(col("count").desc()).show()

24/01/04 19:05:36 ERROR Executor: Exception in task 3.0 in stage 43.0 (TID 66)9]
java.io.FileNotFoundException: /tmp/blockmgr-d9597589-85af-4498-8d41-96b03b89b90f/34/shuffle_13_66_0.index.a54057c5-3c97-4f3d-943e-749249da9b71 (No such file or directory)
	at java.base/java.io.FileOutputStream.open0(Native Method)
	at java.base/java.io.FileOutputStream.open(FileOutputStream.java:298)
	at java.base/java.io.FileOutputStream.<init>(FileOutputStream.java:237)
	at java.base/java.io.FileOutputStream.<init>(FileOutputStream.java:187)
	at org.apache.spark.shuffle.IndexShuffleBlockResolver.writeMetadataFile(IndexShuffleBlockResolver.scala:455)
	at org.apache.spark.shuffle.IndexShuffleBlockResolver.writeMetadataFileAndCommit(IndexShuffleBlockResolver.scala:390)
	at org.apache.spark.shuffle.sort.io.LocalDiskShuffleMapOutputWriter.commitAllPartitions(LocalDiskShuffleMapOutputWriter.java:118)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:141)
	a

Py4JJavaError: An error occurred while calling o780.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 43.0 failed 1 times, most recent failure: Lost task 3.0 in stage 43.0 (TID 66) (okeanos-master executor driver): java.io.FileNotFoundException: /tmp/blockmgr-d9597589-85af-4498-8d41-96b03b89b90f/34/shuffle_13_66_0.index.a54057c5-3c97-4f3d-943e-749249da9b71 (No such file or directory)
	at java.base/java.io.FileOutputStream.open0(Native Method)
	at java.base/java.io.FileOutputStream.open(FileOutputStream.java:298)
	at java.base/java.io.FileOutputStream.<init>(FileOutputStream.java:237)
	at java.base/java.io.FileOutputStream.<init>(FileOutputStream.java:187)
	at org.apache.spark.shuffle.IndexShuffleBlockResolver.writeMetadataFile(IndexShuffleBlockResolver.scala:455)
	at org.apache.spark.shuffle.IndexShuffleBlockResolver.writeMetadataFileAndCommit(IndexShuffleBlockResolver.scala:390)
	at org.apache.spark.shuffle.sort.io.LocalDiskShuffleMapOutp

In [8]:
join_3.orderBy(col("count").desc()).show()

+--------------------+-----+
|        Vict_Descent|count|
+--------------------+-----+
|               White|  312|
|               Other|  102|
|Hispanic/Latin/Me...|   53|
|             Unknown|   26|
|         Other Asian|   16|
|               Black|   14|
+--------------------+-----+



## test 

In [1]:
from pyspark.sql import SparkSession

# Αρχικοποίηση Spark Session
spark = SparkSession.builder \
    .appName("DataFrame Query Example") \
    .master("local[4]") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/05 12:16:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df1 = spark.range(1, 1000001).toDF("id")
df2 = df1.repartition(5)

In [4]:
from pyspark.sql.functions import col

df1 = df1.withColumn("id", col("id") + 5)
df2 = df2.withColumn("id", col("id") + 5)

In [6]:
df1.show()
df2.show()

+---+
| id|
+---+
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
| 20|
| 21|
| 22|
| 23|
| 24|
| 25|
+---+
only showing top 20 rows



+------+
|    id|
+------+
|237712|
| 96795|
| 38616|
|103184|
|169209|
|217424|
| 65306|
| 16031|
|100688|
|128392|
| 99182|
|181581|
| 37125|
| 48765|
|208796|
|227225|
|205712|
|195359|
| 78967|
| 61743|
+------+
only showing top 20 rows



In [ ]:
spark.stop()